examples from https://morvanzhou.github.io/

should use coldfoding to read

In windows, you need to define your job(worker) in a separate .py file and import it. 
Or add task into temporary file, then import it.

In [38]:
%reload_ext autoreload
%autoreload 2

# threading

# multiprocess

In [1]:
import threading as td
import multiprocessing as mp
import inspect


def job(a, d):
    print('aaaaa')

if __name__ == '__main__':
    # t1 = td.Thread(target=job, args=(1, 2))
    p1 = mp.Process(target=job, args=(1, 2))

    # t1.start()
    p1.start()

    # t1.join()
    p1.join()

## Queue

In [3]:
%%writefile tmp_task.py
def job(q):
    res = 0
    for i in range(1000):
        res += i+i**2+i**3
    q.put(res)  # queue

Overwriting tmp_task.py


In [4]:
import multiprocessing as mp
from tmp_task import job


q = mp.Queue()

p1 = mp.Process(target=job, args=(q,))
p2 = mp.Process(target=job, args=(q,))

p1.start()
p2.start()

p1.join()
p2.join()

res1 = q.get()
res2 = q.get()

print(res1+res2)

499667166000


## effectiveness

In [19]:
%%writefile tmp_task.py
# auto create from jupyter
def job(q):
    res = 0
    for i in range(1000000):
        res += i+i**2+i**3
    q.put(res)  # queue

Overwriting tmp_task.py


In [11]:
import multiprocessing as mp
import time
import threading as td

from tmp_task import job

In [16]:
def multicore():
    q = mp.Queue()
    cores = 5
    processes = [mp.Process(target=job, args=(q, )) for i in range(cores)]

    for i in range(cores):
        processes[i].start()

    for i in range(cores):
        processes[i].join()

    res = sum([q.get() for i in range(cores)])
    print(res)


def multithread():
    q = mp.Queue()  # thread一樣可以放進這
    num_threads = 5
    threads = [td.Thread(target=job, args=(q, )) for i in range(num_threads)]

    for i in range(num_threads):
        threads[i].start()

    for i in range(num_threads):
        threads[i].join()

    res = sum([q.get() for i in range(num_threads)])
    print(res)


def normal():
    res = 0
    for _ in range(5):
        for i in range(1000000):
            res += i + i**2 + i**3
    print(res)

In [18]:
st = time.time()

normal()
st1 = time.time()
print(f'normal time: {st1 - st:.4f}\n')

multithread()
st2 = time.time()
print(f'multithread time: {st2 - st1:.4f}\n')

multicore()
print(f'multicore time: {time.time() - st2:.4f}\n')

1249999166667916665000000
normal time: 2.7308

1249999166667916665000000
multithread time: 2.8324

1249999166667916665000000
multicore time: 0.6748



## pool

In [23]:
%%writefile tmp_task.py

# auto create from jupyter
def job(x):
    return x ** 2 

Overwriting tmp_task.py


In [24]:
import multiprocessing as mp

from tmp_task import job

In [37]:
def multicore():
    pool = mp.Pool(processes=10)  # use 10 cores
    
    # map
    res = pool.map(job, range(10))
    print(res)
    
    # apply one stuff
    res = pool.apply_async(job, (2, ))
    print(res.get())
    
    multi_res = [pool.apply_async(job, (i, )) for i in range(10)]
    print([multi_res[i].get() for i in range(10)])
multicore()

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]
4
[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]
